In [ ]:
#!/usr/bin/env python
import feedparser
import os, string
import sys
import tweepy, time
from priv_access import * ## change `priv_access` to `access` with your API tokens
import schedule
import time


## list of the distribution
# mylist_id = '1321018474639085578' #todo add covid example
## list of user id to check
# user_id=''
## reosted error to ignore for the log.list
IGNORE_ERRORS = [327]


# Setup API:
def twitter_setup():
    # Authenticate and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API access:
    api = tweepy.API(auth,parser=tweepy.parsers.JSONParser())
    
#     api = tweepy.API(auth,parser=tweepy.parsers.JSONParser())
    return (api)


class Settings:
    """Twitter bot application settings.

    Enter the RSS feed you want to tweet, or keywords you want to retweet.
    """
    # RSS feeds to read and post tweets from.
    feed_urls = ["https://pubmed.ncbi.nlm.nih.gov/rss/search/1Di1IZzM0R4FRnKYsI1qINYHDYUiWSVAWo0rd3bhufn34wQ9HU/?limit=100&utm_campaign=pubmed-2&fc=20201028084526",
     'http://export.arxiv.org/api/query?search_query=all:psilocybin*&start=0&max_results=100&sortBy=lastUpdatedDate&sortOrder=descending',
        ]
    # nested list/dictionary comprehension to parse multiple RSS feeds
    combined_feed = [feedparser.parse(url) for url in feed_urls]    # Log file to save all tweeted RSS links (one URL per line).
    posted_urls_output_file = "posted-urls.log"

    # Log file to save all retweeted tweets (one tweetid per line).
    posted_retweets_output_file = "posted-retweets.log"

    # Include tweets with these words when retweeting.
    retweet_include_words = ["Drugpolicy", "regulatestimulants", "safeconsumption"]

    # Do not include tweets with these words when retweeting.
    retweet_exclude_words = []


def compose_message(item: feedparser.FeedParserDict) -> str:
    """Compose a tweet from an RSS item (title, link, description)
    and return final tweet message.

    Parameters
    ----------
    item: feedparser.FeedParserDict
        An RSS item.

    Returns
    -------
    str
        Returns a message suited for a Twitter status update.
    """
    title = ' '.join(
        [f"#{x}" if x.lower().translate(str.maketrans('', '', string.punctuation)) in add_hashtag else x for x in
         item["title"].split()])
    link, _ = item["link"], item["description"]
    message = shorten_text(title, maxlength=250) + " " + link
    return message


def shorten_text(text: str, maxlength: int) -> str:
    """Truncate text and append three dots (...) at the end if length exceeds
    maxlength chars.

    Parameters
    ----------
    text: str
        The text you want to shorten.
    maxlength: int
        The maximum character length of the text string.

    Returns
    -------
    str
        Returns a shortened text string.
    """
    return (text[:maxlength] + '...') if len(text) > maxlength else text


def post_tweet(message: str):
    """Post tweet message to account.

    Parameters
    ----------
    message: str
        Message to post on Twitter.
    """
    try:
        twitter_api = twitter_setup()
#         twitter_api.update_status(status=message)
    except tweepy.TweepError as e:
        print(e)


def read_rss_and_tweet(url: str):
    """Read RSS and post feed items as a tweet.

    Parameters
    ----------
    url: str
        URL to RSS feed.
    """
    feeds = Settings.combined_feed
    count = 0
    for feed in feeds:
        if feed:
            while count in range(0, len(feed["items"])):
                item = feed["items"][count]
                link = item["link"]
                link_id = item.id

                if not is_in_logfile(link_id, Settings.posted_urls_output_file):
    #                 post_tweet(message=compose_message(item))
                    write_to_logfile(link_id, Settings.posted_urls_output_file)
                    print("Posted:", link_id, compose_message(item))
                    break
                else:
                    print(count, "Already posted:", link_id, "Trying next")
                    count += 1
        else:
            print("Nothing found in feed", url)


def get_query() -> str:
    """Create Twitter search query with included words minus the
    excluded words.

    Returns
    -------
    str
        Returns a string with the Twitter search query.
    """
    include = " OR ".join(Settings.retweet_include_words)
    exclude = " -".join(Settings.retweet_exclude_words)
    exclude = "-" + exclude if exclude else ""
    return include + " " + exclude


def try_retweet(twitter_api, tweet_text, tweet_id):
    
    '''try to retweet, if already retweeted try next fom the list
    of recent tweets'''

    if not is_in_logfile(
            tweet_id, Settings.posted_retweets_output_file):
        try:
#             twitter_api.retweet(id=tweet_id)
            write_to_logfile(
                tweet_id, Settings.posted_retweets_output_file)
            print("Retweeted {} (id {})".format(shorten_text(
                tweet_text, maxlength=140), tweet_id))
            return True
        except tweepy.TweepError as e:
            if e.api_code in IGNORE_ERRORS:
                return False
            else:
                print(e)
                return True
    else:
        print("Already retweeted {} (id {})".format(
            shorten_text(tweet_text, maxlength=140), tweet_id))


def search_and_retweet(flag='global_search', count=10):
    """Search for a query in tweets, and retweet those tweets.

    Parameters
    ----------
    flag: str
        A query to search for on Twitter. it can be `global_search` to search globally
        or `list_search` reduced to a list defined on mylist_id
    count: int
        Number of tweets to search for. You should probably keep this low
        when you use search_and_retweet() on a schedule (e.g. cronjob).
    """
    try:
        twitter_api = twitter_setup()
        #         search_results = twitter_api.search(q=query, count=count)
        if flag == 'global_search':
            ## search results retweets globally forgiven keywords
            search_results = twitter_api.search(q=get_query(), count=count)  ## standard search results
        else:
            ## search list retwwets most commented ad rt from the experts lists
            search_results = twitter_api.list_timeline(list_id=mylist_id)  ## list to tweet from

    except tweepy.TweepError as e:
        print(e.reason)
        return

    # Make sure we don't retweet any dubplicates.
    count = 0
    ## get the most faved+ rtweeted and retweet it
    max_val = sorted(([(x.retweet_count, x.id_str, x.text) for x in search_results]))

    while (True):
        tweet_id = max_val[-1 - count][1]
        tweet_text = max_val[-1 - count][2]
        if try_retweet(twitter_api, tweet_text, tweet_id):
            break
        elif count > len(search_results):
            print('no more tweets to publish')
            break
        else:
            count += 1
            time.sleep(2)
            continue


def is_in_logfile(content: str, filename: str) -> bool:
    """Does the content exist on any line in the log file?

    Parameters
    ----------
    content: str
        Content to search file for.
    filename: str
        Full path to file to search.

    Returns
    -------
    bool
        Returns `True` if content is found in file, otherwise `False`.
    """
    if os.path.isfile(filename):
        with open(filename) as f:
            lines = f.readlines()
        if (content + "\n" or content) in lines:
            return True
    return False


def retweet_own():
    """
    re-tweet self last tweetet message.
    """
    count = 0
    try:
        while True:
            twitter_api = twitter_setup()
            tweet = twitter_api.user_timeline(id=twitter_api, count=10)[count]
            if not tweet.retweeted:
#                 twitter_api.retweet(tweet.id_str)
                print("retweeted: ", tweet.text)
                break
            else:
                print('already retweeted, trying next')
                count += 1

    except tweepy.TweepError as e:
        print(e)


def write_to_logfile(content: str, filename: str):
    """Append content to log file, on one line.

    Parameters
    ----------
    content: str
        Content to append to file.
    filename: str
        Full path to file that should be appended.
    """
    try:
        with open(filename, "a") as f:
            f.write(content + "\n")
    except IOError as e:
        print(e)


    while 1:
        schedule.run_pending()
        time.sleep(1)


def display_help():
    """Show available commands."""
    print("Syntax: python {} [command]".format(sys.argv[0]))
    print()
    print(" Commands:")
    print("    rss    Read URL and post new items to Twitter")
    print("    rtg    Search and retweet keywords from global feed")
    print("    rtl    Search and retweet keywords from list feed")
    print("    rto    Retweet last own tweet")
    print("    help   Show this help screen")


if __name__ == "__main__":
    if len(sys.argv) > 1:
        if sys.argv[1].lower() == "rss":
            read_rss_and_tweet(url=Settings.combined_feed)
        elif sys.argv[1].lower() == "rtg":
            search_and_retweet('global_search')
        elif sys.argv[1].lower() == "rtl":
            search_and_retweet('list_search')
        elif sys.argv[1].lower() == "rto":
            retweet_own()
        else:
            display_help()
    else:
        display_help()



### Insert user data

In [ ]:
import MySQLdb.cursors
import pandas as pd

twitter_handles=pd.read_csv('TwitterHandles.csv')
display(twitter_handles.head())
## database connection
db = MySQLdb.connect(host='',
                               user='hackathon',
                               port=(3306),
                               password='',
                               database='')

c = db.cursor()

def fill_database(user):

    # IPCC_CH
    # drug_papers
    # GretaThunberg

    tables_list=['new_retweet', 'new_tweet', 'new_user']



    api=twitter_setup()
    tweets = api.user_timeline(screen_name=user, 
                               # 200 is the maximum allowed count
                               count=200,
                               include_rts = True,
                               # Necessary to keep full_text 
                               # otherwise only the first 140 words are extracted
                               tweet_mode = 'extended'
                               )
    user_data = api.get_user(user) 
    
    created_at=user_data['created_at']
    print(created_at)
    user_id=user_data['id']


    tables_dic={}
#     dictionary for the columns insert statement 
    for table in tables_list:
        sql = f"DESCRIBE {table};"
        c.execute(sql)
        fetch_cols = c.fetchall()
        ## keep columns that are in the MySQL table
        column_names = [x[0] for x in fetch_cols]

        cols = ",".join([str(i) for i in column_names])
        tables_dic[table]=cols

    insert_vals=(user_id, user,'',created_at)
    sql = f'INSERT IGNORE INTO new_user ({tables_dic["new_user"]}) VALUES (%s,%s,"%s",%s);'
    # print((sql%insert_vals))
#     c.execute(sql,insert_vals)

    for tweet in tweets:
        
        ## tweet variables
        created_at=tweet['created_at']
        tweet_id=tweet['id']  ## tweet_id
#         is_rt=tweet['retweeted'] ### does not work, gotta check why but 
        if 'retweeted_status' in tweet:  ## but this works!
            is_rt=True
        else: is_rt=False
            
        tweet_text=tweet['full_text']  ## message of the tweet


        
        ## user variables
        user_id=tweet['user']['id']
#         created_at = user_id.created_at

        if 'media' in tweet['entities']:
            tweet_url=(tweet['entities']['media'][0]['url'])
        else:
            tweet_url=''


        ## table tweet#############################
#         print(created_at)
        insert_vals_sql_tweet=(tweet_id,tweet_text,user_id,created_at,tweet_url,is_rt,user)
        sql_tweet = f'INSERT IGNORE INTO new_tweet ({tables_dic["new_tweet"]}) VALUES (%s,"%s",%s,"%s","%s",%s,%s);'

#         print(sql_tweet% insert_vals_sql_tweet)
        c.execute(sql_tweet,insert_vals_sql_tweet)

#         if is_rt == 1:

        if 'retweeted_status' in tweet:

            original_author=tweet['retweeted_status']['user']['id_str']
            retweeted__id=tweet['retweeted_status']['id_str']
#         else:
#             original_author=user_id
#             retweeted__id='?'

            insert_vals_rt=(tweet_id, 0, user_id, original_author)
            sql_retweet = f'INSERT IGNORE INTO new_retweet ({tables_dic["new_retweet"]}) VALUES (%s, %s,%s,%s);'
    #         print(sql% insert_vals_rt)
#             c.execute(sql_retweet,insert_vals_rt)



    db.commit()


In [ ]:
# fill in that db!!
print(tweet['entities'])

count=0

for i, row in twitter_handles.iterrows():
    count+=1
    import_user=(row[1][1:])
    print(import_user)
    fill_database(import_user)
#     if count == 3: break
        
        
## for later 
# import datetime

# date_time_str = 'Mon Jul 27 12:47:02 +0000 2009'
# date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%b-%d %H:%M:%S.%f')

# date_time_str = 'Jun 28 2018 7:40AM'
# date_time_obj = datetime.datetime.strptime(date_time_str, '%b %d %Y %I:%M%p')


## Autoreply Bot 

In [8]:
import csv, time
import tweepy
import json,ast
from access_t import * ## change `priv_access` to `access` with your API tokens
import subprocess

IGNORE_ERRORS = [327]

# Setup API:
def twitter_setup():
    # Authenticate and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    # Return API access:
    api = tweepy.API(auth,parser=tweepy.parsers.JSONParser())
    return (api)


class MyStreamListener(tweepy.StreamListener):
        
    def on_data(self, data):
        # Decode the JSON data
        tweet = json.loads(data)
        # Print out the Tweet
        print((tweet['user']['screen_name'], tweet['text']))

        with open('hold_that_tweet.csv','a') as f:
                f.write(data)
#         except:
#             pass

    def on_error(self, status):
        print(status)

        
        
api=twitter_setup()
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

# if __name__ == '__main__':
#     listener = PrintListener()

# Using readlines() 

# count = 0
# Strips the newline character 

def auto_replies(twit_log_file):
    file1 = open(twit_log_file, 'r') 
    Lines = file1.readlines() 
    
    ## add for Paul's tool
    proc = subprocess.Popen(['echo', '5'], stdout=subprocess.PIPE)
    score = int(proc.stdout.read().decode("utf-8"))
    print(score)

    for line in Lines: 
        tweet=(json.loads(line)) 
        if not 'quoted_status' in tweet:
            continue
        replied_to=(tweet['quoted_status']['user']['screen_name'])
        answer_user=(tweet['user']['screen_name'])
        answer_id=tweet['id']
        
        print(answer_user,replied_to,score)

        # set the scores here
        if score <= 5:
            update_status=f'@{answer_user} no way that is an expert!,do not trust that account!! @{replied_to} has a score of {score} in our database to learn more visit bit.ly/test'

        elif score > 5:
            update_status=f'@{answer_user} be cautious!, this is a neutral! @{replied_to} has a score of {score} in our database to learn more visit bit.ly/test'

        else:
            update_status=f'@{answer_user} this is an expert you can trust!\n@{replied_to} has a score of {score} in our database to learn more visit bit.ly/test'   
        try:

            api.update_status(update_status,answer_id)
            print(update_status)

        except tweepy.TweepError as e:
            continue

myStream.filter(track=['@isthisanexpert'], is_async=True)

while 1:
    try:
        time.sleep(3)
        auto_replies('hold_that_tweet.csv')
        time.sleep(10)
        file = open('hold_that_tweet.csv','r+')
        file.truncate(0)
        file.close()
    except tweepy.TweepError as e:
        pass



420
5
5
('aachsooo', '@isthisanexpert')
('aachsooo', '@isthisanexpert')
5
aachsooo BBCWorld 5
@aachsooo no way that is an expert!,do not trust that account!! @BBCWorld has a score of 5 in our database to learn more visit bit.ly/test
aachsooo BBCWorld 5
5
5
420
5
5
5
5
5
5
5
5
403
5
420
5
5
5
5
5
5
5
5
5
5
5
('JeanMarieFaliss', '@isthisanexpert')
('JeanMarieFaliss', '@isthisanexpert')
5
('OP13325050', '@isthisanexpert')
('OP13325050', '@isthisanexpert')
5
5
5
5
5
5
5
420
5


KeyboardInterrupt: 

In [10]:
update_status="""@{0} this is an expert you can trust!
@{1} has a score of {2}in our database.
to learn more visit bit.ly/test""".format('answer_user','replied_to',"$SCORE")



update_status

# file = open('hold_that_tweet.csv','r+')
# file.truncate(0)
# file.close()

myStream.disconnect()

403
403
403
403
('ySanseacabo', '@poertner_hans @IsThisAnExpert')('ySanseacabo', '@poertner_hans @IsThisAnExpert')

403


In [ ]:
# print(test_id)

api.update_status(f'{"@IsThisAnExpert0"} this is an expert you can trust!\n{"$NAME"} has a score of {"$SCORE"}in our database\nto learn more visit bit.ly/awesomebot',
                  1322702628967616512
                 )
# api.update_status('@ySanseacabo this is an expert you can trust!\n{} has a score of X in our database\nto learn more visit bit.ly/awesomebot',
#                   '1322645929967702018')

In [ ]:
api=twitter_setup()
tweets = api.user_timeline(screen_name=user, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = True,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
user_data = api.get_user('ernafies') 

for tweet in tweets:
    if 'retweeted_status' in tweet:
        print(type(tweet['created_at']))
        print(tweet)
        print()

    

In [ ]:
len('1322458847273979905')

In [ ]:
# search_and_retweet('global_search')

In [ ]:
# read_rss_and_tweet(url=Settings.feed_url)